In [ ]:
#use the Q2 2016 data first
import sys
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt
import datetime

%load_ext autoreload
%autoreload 2

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch import optim
import numpy as np

class SimpleRNN(nn.Module):
    def __init__(self, hidden_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.inp = nn.Linear(5, hidden_size)
        #self.inp = nn.Linear(1, hidden_size)
        #self.rnn = nn.RNN(hidden_size, hidden_size, nonlinearity='tanh')
        self.rnn = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, 1)

    def step(self, input, hidden=None):
        input = self.inp(input.view(1, 5)).unsqueeze(1) #-1 means unsure of number of rows
        #input = self.inp(input.view(1,-1)).unsqueeze(1)
        output, hidden = self.rnn(input, hidden)
        output = self.out(output.squeeze(1))
        return output, hidden
    
    def forward(self, inputs, hidden=None, force=True, steps=0):
        steps = len(inputs)
        outputs = Variable(torch.zeros(steps, 1, 1)) #.cuda()
        for i in range(steps):
            input = inputs[i,:]
            #input = inputs[i]
            output, hidden = self.step(input, hidden)
            outputs[i] = output
        return outputs, hidden

    """
    def forward(self, inputs, hidden=None, force=True, steps=0):
        if force or steps == 0: steps = len(inputs)
        outputs = Variable(torch.zeros(steps, 1, 1))
        for i in range(steps):
            if force or i == 0:
                input = inputs[i,:]
            else:
                input = output
            output, hidden = self.step(input, hidden)
            outputs[i] = output
        return outputs, hidden
    """

n_epochs = 500
n_iters = 50
hidden_size = 30*24

model = SimpleRNN(hidden_size) #.cuda()
#criterion = nn.SmoothL1Loss()
criterion = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

losses = np.zeros(n_epochs) # For plotting
val_errs = np.zeros(n_epochs)

In [ ]:
for epoch in range(n_epochs):

    for iter in range(n_iters):
        #_inputs = sample(50)
        inputs = torch.from_numpy(ex_train[:-1,:]).float()
        inputs = Variable(inputs) #cuda()
        targets = torch.from_numpy(ex_train[1:,0]).float()
        targets = Variable(targets) #.cuda()

        # Use teacher forcing 50% of the time
        force = np.random.random() < 0.1
        outputs, hidden = model(inputs, None, force)

        optimizer.zero_grad()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        losses[epoch] += loss.data[0]
        
        #validation error
        input_val = ex_val[:-1,:]
        target_val = ex_val[1:,0]
        val_inputs = Variable(torch.from_numpy(input_val)).float() #.cuda()
        val_targets = Variable(torch.from_numpy(target_val)).float() #.cuda()
        val_outputs, hidden = model(val_inputs, hidden)
        val_loss = criterion(val_outputs, val_targets)
        
        val_errs[epoch] += val_loss.data[0]

    if epoch > 0:
        print(epoch, "Training loss: " + str(loss.data[0]))
        
        #train error
        print(epoch, "Val loss: " + str(val_loss.data[0]))